# Proyecto Final

## Generacion de Arte

#### Manuel Rodas 21509
#### Sebastian Solorzano 
#### Jose Santisteban

In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


In [22]:
# Configuración
IMG_HEIGHT = 64
IMG_WIDTH = 64
IMG_CHANNELS = 3
LATENT_DIM = 100
EPOCHS = 10000
BATCH_SIZE = 64
DATASET_PATH = './dataset'

In [23]:
# Preprocesamiento del dataset
def load_images(dataset_path):
    images = []
    for file_name in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file_name)
        img = load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = img_to_array(img)
        img_array = (img_array - 127.5) / 127.5  # Normalizar a rango [-1, 1]
        images.append(img_array)
    return np.array(images)

images = load_images(DATASET_PATH)

In [24]:
# Creación del Generador
def build_generator():
    model = tf.keras.Sequential([
        Dense(8 * 8 * 256, input_dim=LATENT_DIM),
        BatchNormalization(),
        LeakyReLU(),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(IMG_CHANNELS, kernel_size=4, strides=2, padding='same', activation='tanh')
    ])
    return model


In [25]:
# Creación del Discriminador
def build_discriminator():
    model = tf.keras.Sequential([
        Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
               kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Dropout(0.3),
        Conv2D(128, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model



In [26]:

# Construcción de las redes
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])



c:\Users\rodas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\rodas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
# Creación del modelo combinado
discriminator.trainable = False
gan_input = tf.keras.Input(shape=(LATENT_DIM,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

In [28]:
# Función para guardar e imprimir imágenes generadas
def save_and_plot_generated_images(epoch, generator, examples=5):
    noise = np.random.normal(0, 1, (examples, LATENT_DIM))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Escalar de vuelta a [0, 1]
    
    plt.figure(figsize=(10, 5))
    for i, img in enumerate(generated_images):
        plt.subplot(1, examples, i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.show()


In [29]:
# Entrenamiento de la GAN
def train_gan(epochs, batch_size):
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        # Entrenar el discriminador
        idx = np.random.randint(0, images.shape[0], batch_size)
        real_images = images[idx]
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        fake_images = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Entrenar el generador
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        g_loss = gan.train_on_batch(noise, real_labels)
        
        # Mostrar progreso
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, D Acc: {d_loss[1]}, G Loss: {g_loss}")
        
        # Guardar e imprimir imágenes cada 1000 épocas
        if epoch % 1000 == 0 and epoch > 0:
            save_and_plot_generated_images(epoch, generator)

In [ ]:
# Entrenar la GAN
train_gan(EPOCHS, BATCH_SIZE)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step 


c:\Users\rodas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, D Loss: 0.5554929971694946, D Acc: 0.625, G Loss: [array(0.6093934, dtype=float32), array(0.6093934, dtype=float32), array(0.46875, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━

Exception ignored in: <function AtomicFunction.__del__ at 0x00000184AF7CE7A0>
Traceback (most recent call last):
  File "c:\Users\rodas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py", line 303, in __del__
    RUNTIME_FUNCTION_REFS.pop(key)
KeyboardInterrupt: 
